In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from collections import Counter
import datetime
import math
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import multi_gpu_model
from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
column_names = ['path', 'make', 'model', 'year_of_production']
raw_dataset = pd.read_csv('/home/ubuntu/Zadanie_3/cars.csv', names = column_names, sep=';',
                      dtype={"path": str, "make": str, 'model': str, 'year_of_production': str})

raw_dataset = raw_dataset[raw_dataset.path != 'path']
raw_dataset = raw_dataset[raw_dataset.year_of_production != 'ints']

train_paths = raw_dataset['path'].values.tolist()
make = raw_dataset['make'].values.tolist()
model = raw_dataset['model'].values.tolist()
year = raw_dataset['year_of_production'].values.tolist()

BATCH_SIZE = 16
IMG_WIDTH = 440
IMG_HEIGHT = 220

In [ ]:
def prepare_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_AREA)
    img = cv2.normalize(img,  img, -0.5, 0.5, cv2.NORM_MINMAX)
    return img

In [ ]:
os.chdir('Car_Dataset')

In [ ]:
CLASS_NAMES_MAKE = np.array(np.unique([name for name in make]))
CLASS_NAMES_MODEL = np.array(np.unique([name for name in model]))
CLASS_NAMES_YEAR = np.array(np.unique([name for name in year]))

In [ ]:
images = []
label_make = []
label_model = []
label_year = []
for i in range(len(train_paths)):
    images.append(prepare_image(train_paths[i]))
    label_make.append(get_label_make(i))
    label_model.append(get_label_model(i))
    label_year.append(get_label_year(i))
images = np.array(images)
label_make = np.array(label_make)
label_model = np.array(label_model)
label_year = np.array(label_year)

In [ ]:
makes = []
for i in range(len(label_make)):
    for j in range(len(label_make[i])):
        if label_make[i][j] == True:
            makes.append(j)
models = []
for i in range(len(label_model)):
    for j in range(len(label_model[i])):
        if label_model[i][j] == True:
            models.append(j)
years = []
for i in range(len(label_year)):
    for j in range(len(label_year[i])):
        if label_year[i][j] == True:
            years.append(j)

In [ ]:
image_input = keras.Input(shape=(220, 440, 3), name='image')
x = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)

output_make = layers.Dense(len(CLASS_NAMES_MAKE), activation='softmax', name='output_make')(x)
output_model = layers.Dense(len(CLASS_NAMES_MODEL), activation='softmax', name='output_model')(x)
output_year = layers.Dense(len(CLASS_NAMES_YEAR), activation='softmax', name='output_year')(x)

model = keras.Model(inputs=image_input, outputs=[output_make, output_model, output_year], name='cars_model')

parallel_model = multi_gpu_model(model, gpus=4) #change to your number of availible gpus

In [ ]:
parallel_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy')

In [ ]:
plot_losses = PlotLossesCallback()
parallel_model.fit(images, [makes, models, years],
                   epochs = 20,
                   batch_size = BATCH_SIZE,
                   callbacks=[plot_losses],
                   validation_split=0.2)